<a href="https://colab.research.google.com/github/tejabalu/HTML-CSS/blob/master/Technin510_L4_PartII_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
from sklearn import svm, metrics
from skimage import io, feature, filters, exposure, color
from sklearn.externals import joblib
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers

# TODO: Copy the code of `FeatureExtractor` class that you wrote 
########################
######## YOUR CODE HERE
########################

class FeatureExtractor:
    
    def __init__(self):
        self.classifier = None
        self.folder = '/content/drive/My Drive/lab4-data/lab4-data/'

    def imread_convert(self, f):
        return io.imread(f).astype(np.uint8)

    def save_classifier(self):
        joblib.dump(self.classifier, self.folder + 'classifier.joblib')

    def load_data_from_folder(self, dir):
        # read all images into an image collection
        ic = io.ImageCollection(self.folder + dir + '*.bmp',
                                load_func=self.imread_convert)

        # create one large array of image data
        data = io.concatenate_images(ic)
        
        # extract labels from image names
        labels = np.array(ic.files)
        for i, f in enumerate(labels):
            m = re.search('_', f)
            labels[i] = (f[len(dir):m.start()]).split('/')[-1]
        
        return(data,labels)

    def extract_image_features(self, data, feature):
        
        
      
        # apply greayscale 
        grayscale = rgb2gray(data)

        # apply gaussian
        filtered_images = filters.gaussian(grayscale)

        # print(filtered_images.shape)

        featured_images = []



        for img in data:
          featured_image = feature(img[:,:,0]).flatten()
          featured_images.append(featured_image)

        featured_images = np.array(featured_images)

        return featured_images


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at /content/drive


`\train` folder contains training+validation images.
`\test` folder contains testing images. 

All images belong to one of the 8 following classes `"drone', 'hands', 'inspection', 'none', 'order', 'place', 'plane', 'truck'`

## Step 1: Load the dataset

In [ ]:
# TODO: Create an object of the class `FeatureExtractor`, call the object `img_clf`
########################
######## YOUR CODE HERE
########################
img_clf 


# load images
(train_val_raw, train_val_labels) = img_clf.load_data_from_folder('train/')
(test_raw, test_labels) = img_clf.load_data_from_folder('test/')

classes = list(np.unique(train_val_labels))
print(f"Classes: {classes}")

Classes: ['drone', 'hands', 'inspection', 'none', 'order', 'place', 'plane', 'truck']


Assign numbers from 0 to 7 for the eight classes that we have. 

0 => drone, 1 => hands, 2 => inspection, 3 => none, 4 => order, 5 => place, 6 => plane, 7 => truck

Let's change string labels in `train_val_labels` and `test_labels` into these numerical labels now. 

In [ ]:
for i in range(len(classes)):
    train_val_labels[train_val_labels == classes[i]] = i
    test_labels[test_labels == classes[i]] = i

train_val_labels = train_val_labels.astype(float)
test_labels = test_labels.astype(float)

## Step 2: Creating TensorFlow datasets
Creating training, validation and testing TensorFlow `Dataset`s from the numpy arrays we have. 

In [ ]:
train_val_dataset = tf.data.Dataset.from_tensor_slices((train_val_raw, train_val_labels)) 
# shuffling the `train+val` dataset before separating them
train_val_dataset = train_val_dataset.shuffle(buffer_size=len(train_val_dataset), seed=42)

VAL_RATIO = 0.1 # Let's use 10% of the `train_val_dataset` for validation
val_len = int(VAL_RATIO * len(train_val_raw))
val_dataset = train_val_dataset.take(val_len)
train_dataset = train_val_dataset.skip(val_len)

test_dataset = tf.data.Dataset.from_tensor_slices((test_raw, test_labels)) 
test_dataset = test_dataset.shuffle(buffer_size=len(test_dataset), seed=42)        

print(f"Train size: {len(train_dataset)}\nVal size: {len(val_dataset)}\nTest size: {len(test_dataset)}")

Train size: 177
Val size: 19
Test size: 40


Batching the Tensorflow datasets:

In [ ]:
BATCH_SIZE = 8
train_dataset_batched = train_dataset.batch(BATCH_SIZE)
val_dataset_batched = val_dataset.batch(BATCH_SIZE)
test_dataset_batched = test_dataset.batch(BATCH_SIZE)

print(f"Train batches: {len(train_dataset_batched)}\nVal batches: {len(val_dataset_batched)}\nTest batches: {len(test_dataset_batched)}")

Train batches: 23
Val batches: 3
Test batches: 5


## Mobile Net v2

A general purpose, deployable computer vision neural network designed by Google that works efficiently for classification, detection and segmentation.  
![](https://miro.medium.com/max/1016/1*5iA55983nBMlQn9f6ICxKg.png)



Bottleneck operator means the Depthwise convolution layer block used in the original MobileNetv2 [paper](https://arxiv.org/pdf/1801.04381.pdf).

## Step 3: Creating the basemodel

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (224, 224)
img_shape = IMG_SIZE + (3,)

# Let's use MobileNetV2 available in Keras as pretrained network.  
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_18[0][0]                   
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

## Step 4: Creating essential layers
Let's define and understand essential layers

### Global Average Pooling

![image](https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-030-34113-8_22/MediaObjects/491550_1_En_22_Fig1_HTML.png)

### Depthwise convolution

Difference between regualar and Depthwise convolution visualized.


Regular convolution

![](https://miro.medium.com/max/778/1*sYpl-7LlrtlOKW8RvlyKOg.png)


Depthwise convolution

![](https://miro.medium.com/max/1038/1*Esdvt3HLoEQFen94x29Z0A.png)

In [ ]:
preprocess_input_fn = tf.keras.applications.mobilenet_v2.preprocess_input 
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(classes))

resize = layers.experimental.preprocessing.Resizing(*IMG_SIZE)

## Step 5: Building the model

Let's build our model using the Keras functional API

In [ ]:
inputs = tf.keras.Input(shape=(240, 320, 3))
inputs_resized = resize(inputs)

# TODO: Preprocess the inputs using the `preprocessed_inputs_fn` defined, and save
#       the results in `preprocessed_inputs` variable
########################
######## YOUR CODE HERE
########################
 

to_gap = base_model(preprocessed_inputs, training=False)

# TODO:  Pass the `to_gap` outputs in the defined `global_average_layer`, and save 
#        the results in `from_gap` variable
########################
######## YOUR CODE HERE
########################


x = tf.keras.layers.Dropout(0.2)(from_gap)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 240, 320, 3)]     0         
_________________________________________________________________
resizing_8 (Resizing)        (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.truediv_9 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_9 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1280)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1280)              0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Step 6: Transfer-learning (training the model)
Training our model for 20 epochs

In [ ]:
EPOCHS = 20

# TODO:  Train the model using  Keras `model.fit`. Use the batched train and val datasets.
########################
######## YOUR CODE HERE
########################


Epoch 1/20
23/23 [==============================] - 7s 189ms/step - loss: 2.3287 - accuracy: 0.1186 - val_loss: 1.9285 - val_accuracy: 0.1053
Epoch 2/20
23/23 [==============================] - 4s 162ms/step - loss: 2.0424 - accuracy: 0.1921 - val_loss: 1.9113 - val_accuracy: 0.2105
Epoch 3/20
23/23 [==============================] - 4s 159ms/step - loss: 1.9327 - accuracy: 0.2599 - val_loss: 1.6698 - val_accuracy: 0.4211
Epoch 4/20
23/23 [==============================] - 4s 160ms/step - loss: 1.7745 - accuracy: 0.2994 - val_loss: 1.4279 - val_accuracy: 0.5789
Epoch 5/20
23/23 [==============================] - 4s 160ms/step - loss: 1.6448 - accuracy: 0.3955 - val_loss: 1.3683 - val_accuracy: 0.6316
Epoch 6/20
23/23 [==============================] - 4s 159ms/step - loss: 1.5402 - accuracy: 0.4576 - val_loss: 1.3253 - val_accuracy: 0.7368
Epoch 7/20
23/23 [==============================] - 4s 158ms/step - loss: 1.4758 - accuracy: 0.4915 - val_loss: 1.1073 - val_accuracy: 0.7895
Epoch 

## Step 7: Testing

In [ ]:
# TODO:  Test the model using  Keras `model.evaluate`. Use the batched test dataset.
#        Print the accuracy
########################
######## YOUR CODE HERE
########################



5/5 [==============================] - 1s 151ms/step - loss: 0.6509 - accuracy: 1.0000
Test accuracy : 1.0


Let's run the trained network on one batch of test-dataset and compare its output with ground-truth labels

In [ ]:
image_batch, label_batch = test_dataset_batched.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).argmax(axis=1)

In [ ]:
print(f"Labels     : {label_batch}\nPredictions: {predictions.astype(float)}")

Labels     : [6. 2. 1. 0. 2. 1. 7. 5.]
Predictions: [6. 2. 1. 0. 2. 1. 7. 5.]
